## Importing necessary libraries

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense,RNN,Flatten,Softmax
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from utility import *
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import pickle
from nltk.translate.gleu_score import sentence_gleu
import math

#### Loading trained model

In [2]:
## code taken from https://colab.research.google.com/drive/172D4jishSgE3N7AO6U2OKAA_0wNnrMOq#scrollTo=mJqOn0snzCRy
## Instantianize model class
model = encoder_decoder(inp_vocab_size,out_vocab_size,embedding_dim,lstm_size,input_length,batch_size,score_fun,att_units)
## as it is custom model we need to complie it 
model.compile(optimizer=optimizer,loss=loss_function)
## This initializes the variables used by the optimizers, as well as any stateful metric variables
model.train_on_batch([incorr_train,corr_train_inp],corr_train_out)
# Load the state of the old model
model.load_weights('attention90low')

#### defining function 1

In [3]:
def function1(input_sentence):
    """
    this function takes incorrect sentence as input and returns correct sentence as output
    """
    
    ## load saved tokenizer
    with open("tokenizer_corr_out_word_attention.pickle","rb") as temp3:
        tokenizer_corr_out = pickle.load(temp3)
    with open("tokenizer_incorr_word_attention.pickle","rb") as temp1:
        tokenizer_incorr = pickle.load(temp1)
    
    ## store word and tokens, as to use it during inference
    corr_dict = tokenizer_corr_out.word_index
    inv_corr = {v: k for k, v in corr_dict.items()}

    ## tokenize input sentence using tokenizer
    input_sentence = tokenizer_incorr.texts_to_sequences([input_sentence])[0]
    
    ## initialize hidden states and cell stated of dencoder
    initial_hidden_state = tf.zeros([1,64])
    initial_cell_state = tf.zeros([1,64])
    encoder_initial_state = [initial_hidden_state,initial_cell_state]
    input_sentence = tf.keras.preprocessing.sequence.pad_sequences([input_sentence],maxlen=25,padding='post')
    input_sentence = input_sentence[0]
    
    ## pass input to encoder and get encoder output, hidden and cell state
    enc_output, enc_state_h, enc_state_c = model.layers[0](np.expand_dims(input_sentence,0),encoder_initial_state)
    
    all = []
    pred = []
    sentence = []
    ## token for start of sentence is 1, so we will pass this to decoder
    cur_vec = np.ones((1, 1),dtype='int')
    
    ## iterate over input sentence
    for i in range(26):
        
        infe_output,deco_state_h, deco_state_c,attention_weights,context_vector = model.layers[1].onestepdecoder(cur_vec, enc_output,enc_state_h, enc_state_c)
        
        ## replace previous hidden and cell state with curent hidden and cell state
        enc_state_h, enc_state_c = deco_state_h, deco_state_c
        
        ## reshaping current word 
        cur_vec = np.reshape(np.argmax(infe_output), (1, 1))
        
        ## if model predicted @(end of sentence) break loop
        if inv_corr[cur_vec[0][0]] == '@':
            break
        temp1 = np.array(infe_output[0])
        
        ## store probablities ate each time stamp in a list
        all.append(temp1)
        
        ## store current output tokens in a list 
        pred.append(cur_vec[0][0])
    for i in pred:
        sentence.append(inv_corr[i])

    y = []
    for i in all:
        y.append(tf.nn.softmax(i).numpy())
    
    ## use beam search decoder 
    z = beam_search_decoder(y, top_k = 1)## 
    results = []
    for i in z:
        sentence = []
        for j in i[0]:
        
            sentence.append(inv_corr[j])
        results.append(" ".join(sentence))
    return  results[0]

#### Testing function 1

In [9]:
result = function1("But today are wednesday")
print("Correct grammatical sentence is -",result)

Correct grammatical sentence is - But today is wednesday


#### defining function 2

In [5]:
def function2(data):
    
    """
    this function takes input data file as input and prints average GLUE score
    """
    test = pd.read_csv(data)
    gleu_score_test = 0
    length = test.shape[0]

    for i in range(length):
        reference = test["correct"].values[i:i+1][0].split()
        predicted = function1(test["incorrect"].values[i:i+1][0])
        split_predicted = [j.split() for j in predicted]
        gleu_score_test = gleu_score_test + sentence_gleu(split_predicted,reference)
    print("Final GLEU Score on Test data are",gleu_score_test/length)

#### testing function 2

In [15]:
function2("test_attention.csv")

Final GLEU Score on Test data are 0.5766407174440898
